In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import boto3
import tempfile
from time import time

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir,'src'))

from d01_data.ingestion_xtdb import ingest_xtdb
from d02_intermediate.clean_xtdb import *
from d00_utils.db_utils import *
from d00_utils.s3_utils import *

db_table = 'metadata'
credentials_file = os.path.join(os.path.expanduser('~'), '.psql_credentials.json')

In [ ]:
io_views = dbReadWriteViews()
io_views.list_tables()

In [ ]:
instances_with_labels_scrappy100 = io_views.get_table('instances_with_labels_scrappy100')
instances_with_labels_scrappy100.head()

In [ ]:
instances_labeled_other_scrappy100 = io_views.get_table('instances_labeled_other_scrappy100')
instances_labeled_other_scrappy100.head()

In [ ]:
prefix_labelled = instances_with_labels_scrappy100['studyidk'].astype(str) + '/a_' + instances_with_labels_scrappy100['instancefilename'].astype(str)

In [ ]:
prefix_other = instances_labeled_other_scrappy100['studyidk'].astype(str) + '/a_' + instances_labeled_other_scrappy100['instancefilename'].astype(str)

In [ ]:
def extract_imgs_from_dicom(bucket, prefix='', suffix='.dcm', outdir=None):
    """
    Get all the keys with a specific suffix from a s3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with this prefix (optional).
    :param suffix: Only fetch objects whose keys end with this suffix, default='dcm'
    :return: list of keys
    """
    s3 = boto3.client('s3')

    keys = []
    kwargs = {'Bucket': bucket}

#    data_dir = os.path.join(projectdir, 'data', '01_raw','dicom_images')
#    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(outdir, exist_ok=True)

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3.list_objects_v2(**kwargs)
        try:
            for obj in resp['Contents']:
                if obj['Key'].startswith(prefix) and obj['Key'].endswith(suffix):
                    keys.append(obj['Key'])

    #                outrawfilename = prefix.split('/')[1]+'_raw'

                    # Download and convert dicom file to raw format
    #                tmp = tempfile.NamedTemporaryFile()
    #                s3.download_file(bucket, obj['Key'], tmp.name)
    #                os.system('gdcmconv -w ' + tmp.name + " " + os.path.join(data_dir, outrawfilename))

                    s3.download_file(bucket, obj['Key'], os.path.join(lablelled_datadir, 
                                                                      'a_' + ''.join(p.split('/a')) + '.dcm'))
        except:
            print(prefix, 'DOWNLOAD ERROR')
        
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

    return keys

In [ ]:
lablelled_datadir = os.path.expanduser('/home/ubuntu/data/01_raw/dcm_sample_labelled')
lablelled_datadir

In [ ]:
other_datadir = os.path.expanduser('/home/ubuntu/data/01_raw/dcm_sample_other')

In [ ]:
for p in prefix_labelled.values:
    print(os.path.join(lablelled_datadir,  'a_' + ''.join(p.split('/a')) + '.dcm'))
    extract_imgs_from_dicom('cibercv', prefix=p, suffix='.dcm', outdir=lablelled_datadir)

In [ ]:
for p in prefix_other.values:
    print(os.path.join(other_datadir, 'a_' + ''.join(p.split('/a')) + '.dcm'))
    extract_imgs_from_dicom('cibercv', prefix=p, suffix='.dcm', outdir=other_datadir)

In [ ]:
for f in get_matching_s3_keys('cibercv', '81864'): print(f)